In [23]:
from typing import Optional, List
from bs4 import BeautifulSoup, Tag
import requests

def get_realtime_price(url: str = 'https://finance.naver.com/sise/sise_index.naver?code=KOSDAQ') :
    req = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
    dom = BeautifulSoup(req.text, "html.parser")
    cur_price = dom.select_one('em#now_value').text
    cur_volume = dom.select_one('td#quant').text
    cur_volume_price = dom.select_one('td#amount').text
    cur_volume_personal = dom.select_one('dl.lst_kos_info').select_one(':nth-child(2)').select_one('span').text
    cur_volume_forg = dom.select_one('dl.lst_kos_info').select_one(':nth-child(3)').select_one('span').text
    cur_volume_instfund = dom.select_one('dl.lst_kos_info').select_one(':nth-child(4)').select_one('span').text

    cur_volume_prog_pos = dom.select_one('dl.lst_kos_info').select_one(':nth-child(6)').select_one('span').text
    cur_volume_prog_neg = dom.select_one('dl.lst_kos_info').select_one(':nth-child(7)').select_one('span').text
    
    print(cur_volume_personal + '|' + cur_volume_forg + '|' + cur_volume_instfund + '|' 
          + cur_volume_prog_pos + '|' + cur_volume_prog_neg)
    print(cur_price + "|" + cur_volume + "|" + cur_volume_price + "|" )
    

get_realtime_price()
    

+270억|-112억|-108억|-31억|-142억
784.20|65,388|490,898|


In [45]:
# 코스닥 종가
from typing import Optional, List
from bs4 import BeautifulSoup, Tag
import requests
from dataclasses import dataclass
from utils.data_type_converter import convert_number

@dataclass
class RealtimePriceCrawl:
    date: str
    closing_price: float
    up_down_per: float
    volume: float
    volume_amt: float

def get_kosdaq_daily_price(page = 1) :
    url = 'https://finance.naver.com/sise/sise_index_day.naver?code=KOSDAQ&page=%s' % page
    req = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
    dom = BeautifulSoup(req.text, "html.parser")
    
    v_date = dom.select('tr > td.date')
    
    res: List[RealtimePriceCrawl] = []
    for v in v_date:
        # print('---------------------------------------')
        # print(v.parent.select_one(':nth-child(1)').text)
        # print(v.parent.select_one(':nth-child(2)').text)
        # print(v.parent.select_one(':nth-child(4)').text.strip())
        # print(v.parent.select_one(':nth-child(5)').text)
        # print(v.parent.select_one(':nth-child(6)').text)
        res.append(RealtimePriceCrawl(
            date=v.parent.select_one(':nth-child(1)').text, 
            closing_price=convert_number(v.parent.select_one(':nth-child(2)').text),
            up_down_per=convert_number(v.parent.select_one(':nth-child(4)').text.strip()),
            volume=convert_number(v.parent.select_one(':nth-child(5)').text),
            volume_amt=convert_number(v.parent.select_one(':nth-child(6)').text)
            )
        )
    return res
    
get_kosdaq_daily_price()

[RealtimePriceCrawl(date='2024.08.20', closing_price=784.18, up_down_per=0.86, volume=430507.0, volume_amt=3436160.0),
 RealtimePriceCrawl(date='2024.08.19', closing_price=777.47, up_down_per=-1.13, volume=1135628.0, volume_amt=7653033.0),
 RealtimePriceCrawl(date='2024.08.16', closing_price=786.33, up_down_per=1.22, volume=1080402.0, volume_amt=8000416.0),
 RealtimePriceCrawl(date='2024.08.14', closing_price=776.83, up_down_per=1.56, volume=1111020.0, volume_amt=7691492.0),
 RealtimePriceCrawl(date='2024.08.13', closing_price=764.86, up_down_per=-1.02, volume=1170514.0, volume_amt=7661966.0),
 RealtimePriceCrawl(date='2024.08.12', closing_price=772.72, up_down_per=1.08, volume=885108.0, volume_amt=6850312.0)]

In [66]:
import pymysql

def save_data(crawled: List[RealtimePriceCrawl] = [], type_code = 'KOSDAQ'):
    conn = pymysql.connect(host='localhost', user='root', password='root', db='zio', charset='utf8mb4')
    cur = conn.cursor()
    
    sql = f"""
        INSERT INTO CRAWL_MARKET_DAILY_DATA(TYPE_CODE, DATEON, CLOSING_PRICE, UP_DOWN_PER, VOLUME, VOLUME_AMT)
        VALUES(%s, %s, %s, %s, %s, %s) as new
        ON DUPLICATE KEY UPDATE 
            TYPE_CODE=new.TYPE_CODE,
            DATEON=new.DATEON,
            CLOSING_PRICE=new.CLOSING_PRICE,
            UP_DOWN_PER=new.UP_DOWN_PER,
            VOLUME=new.VOLUME,
            VOLUME_AMT=new.VOLUME_AMT
    """
    
    for row in crawled:
        try:
            cur.execute(sql, (type_code, row.date, row.closing_price, row.up_down_per, row.volume, row.volume_amt))
        except Exception as e :
            print('duplicated key error --> ' + str(e))
    
    conn.commit()
    conn.close()



In [68]:
crawled = get_kosdaq_daily_price()
print('crawled : %s' % len(crawled))
save_data(crawled, 'KOSDAQ')



crawled : 6


['/Users/a1000074/Library/Application Support/JetBrains/IntelliJIdea2024.1/plugins/python/helpers-pro/jupyter_debug',
 '/Users/a1000074/Library/Application Support/JetBrains/IntelliJIdea2024.1/plugins/python/helpers/pydev',
 '/Users/a1000074/dev/IdeaProjects/myFirstPyCrawlNew/crawl',
 '/Users/a1000074/dev/IdeaProjects/myFirstPyCrawlNew',
 '/opt/miniconda3/envs/myFirstPyCrawlNew/lib/python39.zip',
 '/opt/miniconda3/envs/myFirstPyCrawlNew/lib/python3.9',
 '/opt/miniconda3/envs/myFirstPyCrawlNew/lib/python3.9/lib-dynload',
 '',
 '/opt/miniconda3/envs/myFirstPyCrawlNew/lib/python3.9/site-packages',
 '/opt/miniconda3/envs/myFirstPyCrawlNew/lib/python3.9/site-packages/setuptools/_vendor']

In [2]:
%load_ext autoreload
%autoreload 2

from utils.data_type_converter import DataConverter

a = DataConverter()
a.convert_number('-20.1%')
# 
# convert_number('-20.1%')
# a = utils.data_type_converter.DataConverter()
# a.convert_number('-20.2')

-20.1